# **Make Trade History**

<reference>
    [Markdown](https://datanetworkanalysis.github.io/2019/10/12/markdown)

## Install & Update Libraries

In [1]:
#!pip install --quiet --upgrade pip requests beautifulsoup4 pandas pd
!pip install --quiet --upgrade pip requests beautifulsoup4 pandas pd matplotlib seaborn selenium

!wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | apt-key add -
!echo 'deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main' | tee /etc/apt/sources.list.d/google-chrome.list
!apt-get update
!apt-get install -y google-chrome-stable

!google-chrome --version

!wget -N https://chromedriver.storage.googleapis.com/88.0.4324.27/chromedriver_linux64.zip

!unzip chromedriver_linux64.zip
!chmod +x chromedriver
!mv -f chromedriver /usr/local/share/chromedriver
!ln -s /usr/local/share/chromedriver /usr/local/bin/chromedriver
!ln -s /usr/local/share/chromedriver /usr/bin/chromedriver
!rm chromedriver_linux64.zip
!apt-get update
!apt-get install -y libgconf-2-4

In [2]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import os
import selenium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
%matplotlib inline
requests.__version__, pd.__version__, np.__version__, sns.__version__

('2.25.1', '1.2.1', '1.19.4', '0.11.1')

In [3]:
import bs4
import matplotlib
bs4.__version__, matplotlib.__version__

('4.9.3', '3.3.4')

## get real fee data of NH

In [4]:
url = "https://www.nhqv.com/WMDoc.action?viewPage=/guestGuide/business/feeBuySell.jsp&ga_referrer=https%3A%2F%2Fwww.google.com%2F"
url

'https://www.nhqv.com/WMDoc.action?viewPage=/guestGuide/business/feeBuySell.jsp&ga_referrer=https%3A%2F%2Fwww.google.com%2F'

In [5]:
html = requests.get(url)
html.raise_for_status()
html.content

b'\r\n\r\n\r\n\r\n\r\n\r\n\r\n<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n<meta http-equiv="Content-Type" content="text/html; charset=EUC-KR">\r\n<meta http-equiv="Cache-Control" content="no-cache" />\r\n<meta http-equiv="X-UA-Compatible" content="IE=Edge" />\r\n<title>NH\xc5\xf5\xc0\xda\xc1\xf5\xb1\xc7 (waiLayout)</title>\r\n<link type="text/css" rel="stylesheet" href="/mug/css/mug.css?2012"/>\r\n<link type="text/css" rel="stylesheet" href="/css/nhqv_main.css"/> <!--  css \xc3\xdf\xb0\xa1 -->\r\n<!--[if IE 7]><link type="text/css" rel="stylesheet" href="/mug/css/ie7fix.css" /><![endif]-->\r\n<!--[if IE 8]><link type="text/css" rel="stylesheet" href="/mug/css/ie8fix.css" /><![endif]-->\r\n<!--[if IE 9]><link type="text/css" rel="stylesheet" href="/mug/css/ie9fix.css" /><![endif]-->\r\n<!--[if lte IE 8]><script src="/js/ie.js"></script><![endif]-->\r\n\r\n\r\n\r\n\r\n\r\n<script language="javascript" type="text/javascript" src="/js/domainSetting.js?ver=20210125"></script>\r\n\r\n<s

In [6]:
soup = BeautifulSoup(html.text, 'html.parser')
soup


<!DOCTYPE html>

<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="no-cache" http-equiv="Cache-Control">
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<title>NH투자증권 (waiLayout)</title>
<link href="/mug/css/mug.css?2012" rel="stylesheet" type="text/css"/>
<link href="/css/nhqv_main.css" rel="stylesheet" type="text/css"/> <!--  css 추가 -->
<!--[if IE 7]><link type="text/css" rel="stylesheet" href="/mug/css/ie7fix.css" /><![endif]-->
<!--[if IE 8]><link type="text/css" rel="stylesheet" href="/mug/css/ie8fix.css" /><![endif]-->
<!--[if IE 9]><link type="text/css" rel="stylesheet" href="/mug/css/ie9fix.css" /><![endif]-->
<!--[if lte IE 8]><script src="/js/ie.js"></script><![endif]-->
<script language="javascript" src="/js/domainSetting.js?ver=20210125" type="text/javascript"></script>
<script language="javascript" src="/mug/js/jquery-1.4.min.js?ver=20210125" type="text/javascript"></script>
<script language="javascript" src

tbody = soup.select_one("#contents > div:nth-child(2) > table > tbody")
trs = tbody.find_all('tr')
columns_=["min","max","expression"]
tds = map(lambda x:x.find_all('td').text, trs)
td = map(lambda x:x.text,trs)
arr = np.array(td)
df = pd.DataFrame(columns=columns_).astpye({
    'min':'int32'
    'max':'int32'
    'expression':'object'
})
td = list(td)[:9]
for i, tset in enumerate(td):
    temp = tset.split('\n')
    textset = temp[1:len(temp)-1]
    money,expression=tuple(textset[2:]) if i == 0 else tuple(textset[0:2])
    money = money.replace("미만","").replace(" ", "")\
                    .replace('백','00').replace('천','000').replace('만','0000')\
                        .replace("억","00000000").replace("원","").replace("이상","")
    money = money.split("~")
    expression = expression.replace('%','')
    expression = expression.replace('원','')
    expression = '*' + expression
    if i == 0:
        df = df.append(pd.DataFrame([[np.nan]+money+[expression]],columns=columns_))
    elif i == len(td)-1:
        df = df.append(pd.DataFrame([money+[np.nan]+[expression]],columns=columns_))
    else:
        df = df.append(pd.DataFrame([money+[expression]],columns=columns_))

## get real time price by beautifulsoup
### problem: program execution speed

In [8]:
dt = datetime.today().strftime("%Y/%m/%d")
dt

'2021/02/01'

In [9]:
code = '051910'
url = f"https://finance.naver.com/item/main.nhn?code={code}"

html = requests.get(url)
html.raise_for_status()
url

'https://finance.naver.com/item/main.nhn?code=051910'

In [10]:
import datetime

stock_code = "005930"
        
url = f"https://finance.naver.com/item/main.nhn?code={stock_code}"
html = requests.get(url)
html.raise_for_status()
    
content = BeautifulSoup(html.content, 'html.parser')
    
stock_div = content.find("div",{"class":"wrap_company"})
stock_name = stock_div.find("a").text

feed = content.find("p",{"class":"no_today"})
price = int(feed.find("span",{"class":"blind"}).text.replace(",",""))

#now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
#KST = timezone('Asia/Seoul')
now = datetime.datetime.utcnow()
#now = KST.localize(now)
#now = now.strftime("%Y-%m-%d %H:%M:%S")
now = pd.to_datetime(now,format="%Y-%m-%d %H:%M:%S", errors='raise')
now_local = now.tz_localize('Asia/Seoul')

stock_data = pd.DataFrame([[now_local, stock_name, price]],
                          columns=["requested_time","name","price"]).astype({'price':"int32"})

stock_data

,requested_time,name,price
0,2021-02-01 05:51:30.788739+09:00,삼성전자,82600


## get real time price by  selenium (????)

import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

code = '005930'
url_ = f"https://finance.daum.net/quotes/A{code}#home"

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
#can't crawl real time data?
driver = webdriver.Chrome(executable_path='chromedriver', options=chrome_options)
driver.get(url=url_)

value = driver.find_elemment_by_css_selector("#boxSummary > div > span:nth-child(1) > span.currentB > span.numB.down > strong")
value.text


## make history

In [88]:
#import useful_function as uf
#import crawler
import os
import pytz

def isFileExist(file_name):
    """
    Check if file( or directory) exists
    if file exists, it returns True.
    """
    return os.path.exists(file_name)

def getRecentFeeData(path="../data/fee/"):
    if isFileExist(path):
        file_list = os.listdir(path)
        if len(file_list) > 0:
            file_name = max(file_list)
            data = pd.read_csv(path+file_name)
            return data
        else:
            raise Exception("No Fee Data File : " + path)
    else:
        raise Exception("Directory not Found : " + path)
        
def checkStockCode(stock_code):
    """ Check whether the stock code exists
    
    Parameters
    ----------
    stock_code : str
        the stock code (ex. "005930")
    """
    url = f"https://finance.daum.net/quotes/A{stock_code}#home"
    
    html = requests.get(url)
    html.raise_for_status()
    
    return "alert" not in html.text

def getCurrentStockData(stock_code):
    """
    Return current data of requested company
    
    Parameters
    ----------
    stock_code : str
        the stock code (ex. "005930")
        
    Return Value
    ----------
    DataFrame
    ex)
    requested_time	                    name	    price
	2021-01-30 05:31:34.130440+09:00	삼성전자	82000
    """
    if not checkStockCode(stock_code):
        raise ValueError("Nonexistent stock code")
        
    url = f"https://finance.naver.com/item/main.nhn?code={stock_code}"
    html = requests.get(url)
    html.raise_for_status()
    
    content = BeautifulSoup(html.content, 'html.parser')
    
    stock_div = content.find("div",{"class":"wrap_company"})
    stock_name = stock_div.find("a").text

    feed = content.find("p",{"class":"no_today"})
    price = int(feed.find("span",{"class":"blind"}).text.replace(",",""))
    
    now = datetime.datetime.utcnow()
    now = pd.to_datetime(now,format="%Y-%m-%d %H:%M:%S", errors='raise')
    now_local = now.tz_localize('Asia/Seoul')

    stock_data = pd.DataFrame([[now_local, stock_name, price]],
                          columns=["requested_time","name","price"]).astype({'price':"int32"})
    
    return stock_data

def getLocalTime(area = "Asia/Seoul"):
    """
    Return local time whose format is "Year-Month-Day Hour:Minute:Second"
    Default area is seoul.
    UTC clock(?) is based on UK, so we need to localize clock
    """
    KST = pytz.timezone(area)
    now = datetime.datetime.utcnow()
    now_local = pytz.utc.localize(now).astimezone(KST)
    return now_local

getLocalTime("Europe/London")

datetime.datetime(2021, 2, 1, 7, 14, 43, 796737, tzinfo=<DstTzInfo 'Europe/London' GMT0:00:00 STD>)

In [40]:
data = pd.DataFrame({'date':pd.date_range("2020-01-03",periods=3),'price':[5,4,3],'quantity':[4,11,2]},columns=["date","price","quantity"])
data.loc[data.idxmax()["date"]]

date        2020-01-05 00:00:00
price                         3
quantity                      2
Name: 2, dtype: object

In [90]:
#여기부터 시작 / 최근 5개까지 거래가 현재보다 앞서서 일어나야 하는거 체크하기
#time_zone관련해서 공부
#https://predictors.tistory.com/28
#https://spoqa.github.io/2019/02/15/python-timezone.html

rng = pd.date_range('2/13/2019 9:30', periods=10, freq='B')
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts1 =ts[:7].tz_localize('Europe/London')
ts2 =ts1[2:].tz_convert('Asia/Seoul')
ts1, ts2, ts1 + ts2

(2019-02-13 09:30:00+00:00   -1.107470
 2019-02-14 09:30:00+00:00   -0.873856
 2019-02-15 09:30:00+00:00    0.916366
 2019-02-18 09:30:00+00:00   -1.134038
 2019-02-19 09:30:00+00:00    1.314939
 2019-02-20 09:30:00+00:00    1.529135
 2019-02-21 09:30:00+00:00    0.327645
 dtype: float64,
 2019-02-15 18:30:00+09:00    0.916366
 2019-02-18 18:30:00+09:00   -1.134038
 2019-02-19 18:30:00+09:00    1.314939
 2019-02-20 18:30:00+09:00    1.529135
 2019-02-21 18:30:00+09:00    0.327645
 dtype: float64,
 2019-02-13 09:30:00+00:00         NaN
 2019-02-14 09:30:00+00:00         NaN
 2019-02-15 09:30:00+00:00    1.832732
 2019-02-18 09:30:00+00:00   -2.268077
 2019-02-19 09:30:00+00:00    2.629879
 2019-02-20 09:30:00+00:00    3.058271
 2019-02-21 09:30:00+00:00    0.655290
 dtype: float64)

In [92]:
import datetime
from pytz import timezone, utc

KST = timezone('Asia/Seoul')
now = datetime.datetime.utcnow()
#utc.localize(now)
#KST.localize(now)
now_local = utc.localize(now).astimezone(KST)

temp_date = datetime.datetime(2021,2,2,1,0,15,881617)
temp_date = utc.localize(temp_date).astimezone(timezone("Asia/Seoul"))

temp_date > now_local

True

In [34]:
#history_file_name = "../data/history/"+"purchase_history_" + "1" + ".csv"
#fd = open(history_file_name,'w')
#fd.close()

data = pd.DataFrame(np.arange(15).reshape((5,3)),columns=[551,233,222])
#data.to_csv(history_file_name,mode='a',header=True,index=False)
data

history = pd.read_csv(history_file_name)
history

,551,233,222
0,0,1,2
1,3,4,5
2,6,7,8
3,9,10,11
4,12,13,14


In [ ]:
import os
#히스토리 쓰는 클래스
#쓰기, 상태 리턴(가지고 있는 주식 개수), 
#csv파일 명은 purchase_history_[strategy_index]
#시간 관련 함수 만들어야 할 듯 utc + 9
#타입 제대로 적기
def getRecentFeeData(path="../data/fee/"):
    if isFileExist(path):
        file_list = os.listdir(path)
        if len(file_list) > 0:
            file_name = max(file_list)
            data = pd.read_csv(path+file_name)
            return data
        else:
            raise FileNotFoundError("No Fee Data File : " + path)
    else:
        raise FileNotFoundError("Directory not Found : " + path)

class history:
    def __init__(self, strategy_id, data_path="../data/history/"):
        self.data_path = data_path
        self.strategy_id = strategy_id
        self.history_file_name = data_path + "purchase_history_" + str(self.strategy_id) + ".csv"
        
        self.fd = open(self.history_file_name,'w') #temp
        self.fd.close()
        
    def ReadHistoryFile(self):
        if isFileExist(self.history_file_name):
            try:
                history = pd.read_csv(self.history_file_name)
                return history
            raise EmptyDataError("No Data to Read : " + history_file_name)
        else:
            raise FilenotFoundError("No History File : " + history_file_name)
    
    def writeHistoryFile(self, purchase_info):
        ''' 
        purchase_info
        dataframe with column [stock_code, quantity]
        can write only one purchase history.
        이거 여러개 쓰는것도 만들어야 하나
        date stock_code stock_name price quantity remain
        quantity에서 판거는 -x 사는거는 x
        '''
        try:
            temp = pd.DataFrame([["temp",0]],columns=["stock_code","quantity"]).astype(
                                    {"stock_code":"object",
                                    "quantity":"int32"}).compare(purchase_info)
        except ValueError:
            raise ValueError("Wrong Input")
        except KeyError:
            raise KeyError("Wrong Input")
        
        if not checkStockCode(purchase_info['stock_code']):
            raise ValueError("Nonexistent stock code")
        
        if not isFileExist(self.history_file_name):
            raise FilenotFoundError("No History File : " + history_file_name)
            
        info = purchase_info.loc[:] #stock_code, quantity
        stock_data = getCurrentStockData(info["stock_code"]) #"requested_time","name","price"
        
        history = pd.read_csv(self.history_file_name)
        remain = data.loc[data.idxmax()['date']]["quantity"] - info["amount"]*stock_data["price"]
        
        data = {"date":stock_data["requested_time"],"stock_code":info["stock_code"],
                    "stock_name":stock_data["name"],"price":stock_data["price"],
                        "quantity":info["quantity"],"remain":remain}
        data = pd.DataFrame(data).astype({
                "remain":"int32"})
        
        fd = open(self.history_file_name,'r')
        is_not_first = bool(len(fd.readline()))
        if remain > 0:
            data.to_csv(self.history_file_name,mode='a',header=is_not_first,index=False)
        else:
            raise Exception("Not Enough Money : " + "isBuyable")
        
    def getStatus():
        #현재 가지고 있는 전체 주식들 정보 / 현재 남은 돈
        
    def getStatus(self, stock_code):
        None
    def isBuyable():
        #잔고
        #최근 구매가 현재 시간을 넘으면 안됨 -> 이거는 함수로 하나 만들자
        None
        
    def isSellable():
        None
        #최근 구매가 현재 시간을 넘으면 안됨
        #가지고 있는 주식개수가 살려는 주식 개수 넘으면 안됨
        
        
class virtualTrade:
    def __init(self, strategy_id):
        self.strategy_id = strategy_id
        self.history = history(strategy_id)
    
        def buy():
            None
        def sell():
            None
        def putMoney():
            None
        